### Import essential libraries

In [4]:
import random
from collections import Counter, defaultdict
import numpy as np
import sys
import random,progressbar, operator
import hashlib, zlib, progressbar
import pandas as pd
import math

### Helper functions

In [5]:
def generate_stream(n, k):
    freqs = [1] * (n - k)
    freqs += [1000] * k
    random.shuffle(freqs)
    stream = []
    for i in range(n):
        stream.append(random.choice(freqs))
    return stream

def generate_stream2(n, k):
    return ['0'] * (n // 2) + [str(i % (k - 1) + 1) for i in range(n // 2)]

def get_zero():
    return 0
def get_exact_count(inputlist):
    res = defaultdict(get_zero)
    for i in inputlist:
        res[i] += 1
    return res

def get_list_from_file(fdir):
    file1 = open(fdir, 'r')
    Lines = file1.read().splitlines()
    return Lines

def get_exact_count(inputlist):
    res = defaultdict(get_zero)
    for i in inputlist:
        res[i] += 1
    return res

def get_top_hitters(x, table = [], inputlist = []):
    exact_count = get_exact_count(inputlist) if table == [] else table
    sorted_table = dict(sorted(exact_count.items(), key=operator.itemgetter(1), reverse=True))
    res = []
    cnt = 0
    for key in sorted_table:
        res.append(key)
        cnt += 1
        if(cnt == x):
            break
    return res

def get_correct_rate(actual, mg, round_digits):
    correct = 0
    for i in mg:
        if(i in actual):
            correct += 1
    return round(correct/len(mg),round_digits)

def extract_words(sentences):
    words = []
    for sentence in sentences:
        sentence_words = sentence.split()
        for word in sentence_words:
            words.append(word.lower())
    unique_words = set(words)

    return words, unique_words, len(unique_words)

### Implementation of Lossy Counting Algorithm

In [12]:
class LossyCountingSketch(object):
    def __init__(self, epsilon):
        self.n = 0
        self.count = defaultdict(int)
        self.bucket_id = {}
        self.epsilon = epsilon
        self.current_bucket_id = 1

    def get_count(self, item):
        return self.count[item]
    
    def get_counts(self):
        return self.count

    def get_bucket_id(self, item):
        return self.bucket_id[item]

    def add(self, item):
        self.n += 1
        if item not in self.count:
            self.bucket_id[item] = self.current_bucket_id - 1
        self.count[item] += 1

        if self.n % math.floor(1 / self.epsilon) == 0:
            self.trim()
            self.current_bucket_id += 1

    def output_with_threshold_rate(self, threshold_rate):
        return self.output(threshold_rate * self.n)

    def trim(self):
        for item in list(self.count.keys()):
            if self.count[item] <= self.current_bucket_id - self.bucket_id[item]:
                del self.count[item]
                del self.bucket_id[item]

    def output(self, threshold_count):
        assert threshold_count > self.epsilon * self.n, "too small threshold"

        self.trim()
        for item, total in self.count.items():
            if total >= threshold_count - self.epsilon * self.n:
                yield item

### Shakespeare Quotes

In [8]:
df = pd.read_csv("Shakespeare_data.csv")
lines = df["PlayerLine"]

words, uniWords, numUniWords = extract_words(lines)

### Network Dataset

In [ ]:
lines = []
with open("realData.txt") as f:
    lines = f.read().splitlines()

flist = get_list_from_file("realData.txt")

exact_count = get_exact_count(flist)
actual_topx = get_top_hitters(25, [], flist)
actual_topx.sort()

### Lyrics Dataset

In [ ]:
df = pd.read_csv("tcc_ceds_music.csv")
lines = df["lyrics"]

words, uniWords, numUniWords = extract_words(lines)

## Space Limitation and 3 datasets

### Network Dataset

In [ ]:
flist = get_list_from_file("realData.txt")
exact_count = get_exact_count(flist)
actual_topx = get_top_hitters(25, [], flist)
actual_topx.sort()
stream = flist
n = 79989

stream = words
result = []
with open("50.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        result.append(int(line.rstrip()))

print(result)
    
percent = []
print("running experiments...")
for ele in result:
    epsilon = 1/ele
    threshold = math.ceil(epsilon*n)
    lcs = LossyCountingSketch(epsilon=epsilon)
    for elem in stream:
        lcs.add(elem)

    lsct = []
    for item in sorted(lcs.output(threshold),
                            key=lambda x: x[1], reverse=True):
        lsct.append(item)

    if(len(lsct) >= 25):
        lcs_correct = len(set(lsct[:25]).intersection(set(actual_topx)))
    else:
        lcs_correct = len(set(lsct).intersection(set(actual_topx[:len(lsct)])))

    percent.append(lcs_correct/len(actual_topx))

print(percent)

with open("lca_result.txt","w") as f:
    for item in percent:
        f.write(str(item)+"\n")

### Shakespeare and Lyrics (commented code is for Shakespeare dataset)  
#### Please comment out the current dataset and uncomment the other one

In [14]:
import pandas as pd

# df = pd.read_csv("Shakespeare_data.csv")
# lines = df["PlayerLine"]

df = pd.read_csv("tcc_ceds_music.csv")
lines = df["lyrics"]

words, uniWords, numUniWords = extract_words(lines)

print(numUniWords)
print(len(words))

result = []
with open("50.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        result.append(int(line.rstrip()))

print(result)

stream = words

# n = 814671
n = 2071963
actual_topx = get_top_hitters(25, [], words)

percent = []
print("running experiments...")
for ele in result:
    epsilon = 1/ele
    threshold = math.ceil(epsilon*n)
    lcs = LossyCountingSketch(epsilon=epsilon)
    for elem in stream:
        lcs.add(elem)

    lsct = []
    for item in sorted(lcs.output(threshold),
                            key=lambda x: x[1], reverse=True):
        lsct.append(item)

    if(len(lsct) >= 25):
        lcs_correct = len(set(lsct[:25]).intersection(set(actual_topx)))
    else:
        lcs_correct = len(set(lsct).intersection(set(actual_topx[:len(lsct)])))

    percent.append(lcs_correct/len(actual_topx))

print(percent)

with open("lca_result.txt","w") as f:
    for item in percent:
        f.write(str(item)+"\n")

51693
2071963
[50, 75, 100, 150, 200, 400, 600, 700, 750, 800, 850, 900, 950, 1000, 1050, 1200, 1300, 1400, 1500, 1700, 1750, 1800, 1850, 1900, 2000, 2500, 3000, 4000, 4350, 4400]
running experiments...
[0.08, 0.16, 0.24, 0.36, 0.32, 0.08, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]


## Biased Dataset

### First 25

In [ ]:
numbers = []
for i in range(1, 10001):
    if i <= 25:
        numbers.extend([i] * 2)
    else:
        numbers.extend([i] * 1)

print(len(numbers))

### Last 25

In [ ]:
numbers = []
for i in range(1, 10001):
    if i >= 9976:
        numbers.extend([i] * 2)
    else:
        numbers.extend([i] * 1)

print(len(numbers))

In [ ]:
exact_count = get_exact_count(numbers)
actual_topx = get_top_hitters(25, [], numbers)
actual_topx.sort()

stream = numbers
epsilon = 0.001
lcs = LossyCountingSketch(epsilon=epsilon)
for elem in stream:
    lcs.add(elem)

lsct = []
print(lcs.n*lcs.epsilon)

threshold = math.ceil(lcs.n*lcs.epsilon)
for item in sorted(lcs.output(threshold),
                            key=lambda x: x[1], reverse=True):
    lsct.append(item)

print("n",lcs.n)
print("epsilon",lcs.epsilon)

print("Number of top hitters: ",len(lsct))

lcs_correct = len(set(lsct[:25]).intersection(set(actual_topx)))

print("\n################## Count-Min Sketch SIMULATION REPORT ##################\n")
print("Using Artificial Data")
print("\n### PARAMETERS ###")
print("Width of bucket (K): " + str(1/epsilon))
print("Maximum Number of bucket:",max(lcs._bucket_id.values()))
print("Comparing Top " + str(25) + " heavy hitters.")
print("Space Used: " + str(len(lcs._bucket_id.values())))
print("\n### RESULT ###")
if (False):
    print("Actual top hitters: ")
    print(actual_topx)
    print("CMS top hitters: ")
    print(lcs_correct)
print("Number of actual top hitters: " + str(len(actual_topx)))
print("Number of LSC top hitters: " + str(len(lsct[:25])))
print("Percentage of corrected identified heavy hitters: " + str(
    lcs_correct/len(actual_topx))
)

LCA paper: https://www.vldb.org/conf/2002/S10P03.pdf  
Datasets:
https://www.kaggle.com/datasets/kingburrito666/shakespeare-plays  
https://www.kaggle.com/datasets/saurabhshahane/music-dataset-1950-to-2019